Создаем 4-слойную нейронную сеть в датасете MNIST. Входной слой состоит из 28*28 = 784 пикселей с оттенками серого, которые составляют входные данные в датасете MNIST. Входные данные далее проходят через два скрытых слоя, каждый из которых содержит 200 узлов, использующих линейную выпрямительную функцию активации (ReLU). Выходной слой с десятью узлами, соответствующими десяти рукописным цифрам от 0 до 9. 

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Чтобы создать нейронную сеть в PyTorch, используется класс nn.Module. Чтобы им воспользоваться, необходимо наследование, что позволит использовать весь функционал базового класса nn.Module, но при этом еще имеется возможность переписать базовый класс для конструирования модели или прямого прохождения через сеть.    


In [5]:
def create_nn(batch_size=250, learning_rate=0.1,
              log_interval=119):

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)
    

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            # создание 3 полносвязных слоев
            self.fc1 = nn.Linear(28 * 28, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 10)

        # метод прохода данных по сети    
        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)

    net = Net()
    print("Structura Seti: ",net)

    # оптимизатор на основе стохастического градиентного спуска, скорость 
    # обучения = 0.1
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # функция потерь - отрицательное логарифмическое правдоподобие
    criterion = nn.NLLLoss()

    # цикл тренировки
    for epoch in range(5):
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data, target = Variable(data), Variable(target)
            # изменим размер (1,28,28)  до 1-мерного случая для 28 х 28 = 784 входных
            # узла.
            # (batch_size, 784)
            
            data = data.view(-1, 28*28)
            # явный перезапуск градиентов
            optimizer.zero_grad()
            # подаём часть данных. net_out - логарифмич. выходной softmax
            net_out = net(data)
            # функция потери отрицательного логарифмического правдоподобия
            loss = criterion(net_out, target)
            # обратное распространение ошибки
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.5f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.data))
       
    # тестирование
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 28 * 28)
        net_out = net(data)
        # суммируем  batch потери
        test_loss += criterion(net_out, target).data
        pred = net_out.data.max(1)[1]  # индекс масимальной лог вероятности
        correct += pred.eq(target.data).sum() # счетчик правильных ответов сети

    # Суммируя выходы функции .eq(), получаем счетчик количества раз, когда нейронная сеть выдает правильный ответ. 
    # По накопленной сумме правильных предсказаний можно определить общую точность сети на тренировочном датасете.
    # Наконец, проходя по каждой партии входных данных, выводим среднее значение функции потери и точность модели:
    test_loss /= len(test_loader.dataset)
    print('nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
        
    elif run_opt == 2:
        create_nn()

Structura Seti:  Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.31425


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [29750/60000 (50%)]	Loss: 0.09859
Train Epoch: 0 [59500/60000 (99%)]	Loss: 0.18816
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.05509
Train Epoch: 1 [29750/60000 (50%)]	Loss: 0.10004
Train Epoch: 1 [59500/60000 (99%)]	Loss: 0.08605
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.03997
Train Epoch: 2 [29750/60000 (50%)]	Loss: 0.09247
Train Epoch: 2 [59500/60000 (99%)]	Loss: 0.09819
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.01412
Train Epoch: 3 [29750/60000 (50%)]	Loss: 0.03850
Train Epoch: 3 [59500/60000 (99%)]	Loss: 0.10513
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.02020
Train Epoch: 4 [29750/60000 (50%)]	Loss: 0.00896
Train Epoch: 4 [59500/60000 (99%)]	Loss: 0.06427


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


nTest set: Average loss: 0.0003, Accuracy: 9796/10000 (98.0%)



На выходе получаем прогресс на протяжении эпох тренировки и ошибку нейросети в этот момент.   
После тренировки сети получаем следующие результаты:  
Средняя потеря: 0,0003, точность: 9796/10000 (98,0%).
